# Assistants function calling with Bing Search
In this notebook, we'll show how you can use the [Bing Search APIs](https://www.microsoft.com/bing/apis/llm) and [function calling](https://learn.microsoft.com/azure/ai-services/openai/how-to/function-calling?tabs=python) to ground Azure OpenAI models on data from the web. This is a great way to give the model access to up to date data from the web.

You'll need to create a [Bing Search resouce](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) before you begin.

### Time

You should expect to spend 10 minutes running this sample.

### Before you begin
#### Installation
The following packages are required to execute this notebook.

In [40]:
#Install the packages\
%pip install requests openai

Note: you may need to restart the kernel to use updated packages.


### Resource config setup
Update the following config to include details of your Azure OpenAI and Bing Search resources

In [41]:
config = {
    "DEPLOYMENT_NAME":"<DEPLOYMENT_NAME>",
    "AOAI_API_BASE":"https://<YOUR_RESOURCE_NAME>.openai.azure.com",
    "AOAI_API_VERSION":"2024-02-15-preview",

    "AOAI_API_KEY":"<AOAI_RESOURCE_API_KEY>",
    "BING_SEARCH_SUBSCRIPTION_KEY": "<BING_SEARCH_SUBSCRIPTION_KEY>"
}

In [42]:
import requests
import json
import time 

from openai import AzureOpenAI

azure_endpoint = config["AOAI_API_BASE"]
api_version = config["AOAI_API_VERSION"]
aoai_api_key= config["AOAI_API_KEY"] 
deployment_name = config['DEPLOYMENT_NAME']  

bing_search_subscription_key = config['BING_SEARCH_SUBSCRIPTION_KEY']
bing_search_url = "https://api.bing.microsoft.com/v7.0/search"

### Define a function to call the Bing Search APIs
To learn more about using the Bing Search APIs with Azure OpenAI, see [Bing Search APIs, with your LLM](https://learn.microsoft.com/bing/search-apis/bing-web-search/use-display-requirements-llm).

In [43]:
def search(query):
    headers = {"Ocp-Apim-Subscription-Key": bing_search_subscription_key}
    params = {"q": query, "textDecorations": False }
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results['webPages']['value']:
        output.append({
            'title': result['name'],
            'link': result['url'],
            'snippet': result['snippet']
        })

    return json.dumps(output)

In [44]:
search("where will the 2032 olympics be held?")

'[{"title": "2032 Summer Olympics - Wikipedia", "link": "https://en.wikipedia.org/wiki/2032_Summer_Olympics", "snippet": "The 2032 Summer Olympics, officially known as the Games of the XXXV Olympiad and also known as Brisbane 2032 is an upcoming international multi-sport event scheduled to take place between 23 July to 8 August 2032, in Brisbane, Queensland, Australia. [1]"}, {"title": "Brisbane 2032 Summer Olympics - Olympic Games, Medals, Results & Latest ...", "link": "https://olympics.com/en/olympic-games/brisbane-2032", "snippet": "Brisbane 2032 23 July - 8 August 3103 days Australia Official website Brisbane 2032 Annual Report 2022-23 Brisbane 2032 | Olympic Games Countdown Begins: Brisbane Celebrates Nine-Year Mark to 2032 Summer Olympics Brisbane 2032 | Olympic Games 01:01 Brisbane 2032 Olympics Marks Nine-Year Milestone with Grand Celebrations"}, {"title": "Brisbane 2032 Olympic venues announced | Austadiums", "link": "https://www.austadiums.com/news/921/brisbane-2032-olympic-

### Get things running end to end
In the following cells, we will define some functions essential for assistants with function calling. All these functions come together in our final cell, where we will define a new web search assistant, give it instructions on its functionality and ask a question.

In [45]:
def poll_run_till_completion(client, thread_id, run_id, available_functions, max_steps=10, wait=3, verbose=True):
    '''
    Poll a run until it is completed or failed or exceeds a certain number of iterations (MAX_STEPS)
    with a preset wait in between polls
    
    @param client: OpenAI client
    @param thread_id: Thread ID
    @param run_id: Run ID
    @param assistant_id: Assistant ID
    @param max_steps: Maximum number of steps to poll
    @param wait: Wait time in seconds between polls
    @param verbose: Print verbose output
    @return: Run object

    '''
    
    if client is None and thread_id is None or run_id is None:
        print("Client, Thread ID and Run ID are required.")
        return None
    try:
        cnt = 0
        while cnt < max_steps:
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
            print("Poll {}: {}".format(cnt, run.status))
            cnt += 1
            if run.status == "requires_action":
                tool_responses = []
                if run.required_action.type == "submit_tool_outputs" and run.required_action.submit_tool_outputs.tool_calls is not None:
                    tool_calls = run.required_action.submit_tool_outputs.tool_calls

                    for call in tool_calls:
                        if call.type == "function":
                            if call.function.name not in available_functions:
                                raise Exception("Function requested by the model does not exist")
                            function_to_call = available_functions[call.function.name]
                            tool_response = function_to_call(**json.loads(call.function.arguments))
                            tool_responses.append({
                                "tool_call_id": call.id,
                                "output": tool_response
                            })

                run = client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id,
                    run_id=run.id,
                    tool_outputs=tool_responses
                )
            if run.status == "failed":
                print("Run failed.")
                break
            if run.status == "completed":
                break
            time.sleep(wait)

        return run
    except Exception as e:
        print(e)
        return None

In [46]:
def create_message(client=None,
                   thread_id=None,
                   role="",
                   content="",
                   file_ids=[],
                   metadata={},
                   message_id=None):
    '''
    Create a message in a thread using the client.

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param role: Message role (user or assistant)
    @param content: Message content
    @param file_ids: Message file IDs
    @param metadata: Message metadata
    @param message_id: Message ID
    @return: Message object

    '''
    if client is None:
        print("Client parameter is required.")
        return None
    try:
        if thread_id is not None:
            if message_id is not None:
                return client.beta.threads.messages.retrieve(thread_id=thread_id, message_id=message_id)
            else:
                if file_ids is not None and len(file_ids) > 0:
                    return client.beta.threads.messages.create(thread_id=thread_id, role=role, content=content, file_ids=file_ids)
                elif metadata is not None and len(metadata) > 0:
                    return client.beta.threads.messages.create(thread_id=thread_id, role=role, content=content, metadata=metadata)
                elif file_ids is not None and len(file_ids) > 0 and metadata is not None and len(metadata) > 0:
                    return client.beta.threads.messages.create(thread_id=thread_id, role=role, content=content, file_ids=file_ids, metadata=metadata)

                return client.beta.threads.messages.create(thread_id=thread_id, role=role, content=content)
        else:
            print("Thread ID is required.")
            return None
    except Exception as e:
        print(e)
        return None

In [47]:
def retrieve_and_print_messages(client, thread_id, out_dir=None, verbose=True):
    '''
    Retrieve a list of messages in a thread and print it out with the query and response

    @param client: OpenAI client
    @param thread_id: Thread ID
    @param out_dir: Output directory to save images
    @return: Messages object

    '''

    if client is None and thread_id is None:
        print("Client and Thread ID are required.")
        return None
    try:
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        display_role = {'user': 'User query', 'assistant': 'Assistant response'}

        prev_role = None
        print("\n\nCONVERSATION:")
        for md in reversed(messages.data):
            if prev_role == 'assistant' and md.role == 'user' and verbose:
                print("------ \n")

            for mc in md.content:
                # Check if valid text field is present in the mc object
                if mc.type == 'text':
                    txt_val = mc.text.value
                # Check if valid image field is present in the mc object
                elif mc.type == 'image_file':
                    image_data = client.files.content(mc.image_file.file_id)

                    if out_dir is not None and os.path.exists(out_dir):
                        with open(os.path.join(out_dir, mc.image_file.file_id+".png"), "wb") as f:
                            f.write(image_data.read()) 
                
                if verbose:
                    if prev_role == md.role:
                        print(txt_val)
                    else:
                        print("{}:\n{}".format(display_role[md.role], txt_val))
            prev_role = md.role
        return messages
    except Exception as e:
        print(e)
        return None

In [48]:
name = "websearch-assistant"
instructions = """You are an assistant designed to help people answer questions.

You have access to query the web using Bing Search. You should call bing search whenever a question requires up to date information or could benefit from web data.
"""

message = {"role": "user", "content": "How tall is mount rainier?"}

                
tools = [  
    {
        "type": "function",
        "function" :
        {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query",
                    }
                },
                "required": ["query"],
            }
        }
    }
]

available_functions = {'search_bing': search}

client = AzureOpenAI(api_key=aoai_api_key, api_version=api_version, azure_endpoint=azure_endpoint)

assistant = client.beta.assistants.create(
                name=name,
                description="",
                instructions=instructions,
                tools=tools,
                model=deployment_name
            )

thread = client.beta.threads.create()
create_message(client, thread.id, message["role"], message["content"])


run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id, instructions=instructions)
poll_run_till_completion(client, thread.id, run.id, available_functions)
messages = retrieve_and_print_messages(client, thread.id)



Poll 0: queued
Poll 1: requires_action
Poll 2: completed


CONVERSATION:
User query:
How tall is mount rainier?
Assistant response:
Mount Rainier is 14,410 feet (4,392 meters) tall.
